<a href="https://colab.research.google.com/github/JISOO0213/MidProject_NewsCategory/blob/main/%EC%A4%91%EA%B0%84%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EB%89%B4%EC%8A%A4_%EA%B8%B0%EC%82%AC_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install sentencepiece # 토크나이저 인스톨

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 18.2 MB/s 


In [ ]:
# imports
import argparse # 자주 사용하게 될 SEED 값이나 경로 등을 딕셔너리 형태로 저장해주는 모듈 
import os  # 운영체제 제어 
import random  # 랜덤
import shutil # 쉘 명령어 등을 대신 실행해주는 유틸 
import json
import zipfile # 압축 푸는 애
import math 
import copy
import collections  # dict 형태의 자료들을 좀더 쉽게 사용할 수 있게 만들어주는 패키지  - counter: 각 dict에 있는 빈도 세어주는 역할
import re  # 정규식에 의해 텍스트 데이터를 찾을 때 사용

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm  # 토크나이저
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange   # 작업량을 막대그래프로 표시 

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds #Dataset API
import numpy as np #Linear Algebra
import matplotlib.pyplot as plt #Data visualization
import os #Manipulate Files
from PIL import Image #Manipulate Images

import warnings
warnings.filterwarnings('ignore') #ignores warnings

#Make sure Tensorflow is version 2.0 or higher
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.9.2


# 1. 파일확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ITStudy/data/

!unzip -qq "/content/drive/MyDrive/ITStudy/data/mid_proj.zip"

/content/drive/MyDrive/ITStudy/data


In [ ]:
%cd /content/drive/MyDrive/ITStudy/04_DL

!unzip -qq "/content/drive/MyDrive/ITStudy/data/[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_test.zip"

/content/drive/MyDrive/ITStudy/04_DL


In [ ]:
%cd /content/drive/MyDrive/ITStudy/04_DL

!unzip -qq "/content/drive/MyDrive/ITStudy/data/[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_train.zip"

/content/drive/MyDrive/ITStudy/04_DL


# 2. 내용확인

In [ ]:
# data 경로 저장
data_dir = '/content/drive/MyDrive/ITStudy/data/'

In [ ]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234
}
args = argparse.Namespace(**args)   # seed 필요하면 여기서 갖고오면 됨

print(args)

Namespace(seed=1234)


In [ ]:
args.seed

1234

In [ ]:
# random seed 설정
random.seed(args.seed)
np.random.seed(args.seed)
tf.random.set_seed(args.seed)

In [ ]:
args.corpus1 = os.path.join(data_dir, "[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_test.zip")
args.corpus2 = os.path.join(data_dir, "[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_train.zip")

In [ ]:
args.corpus1

'/content/drive/MyDrive/ITStudy/data/[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_test.zip'

In [ ]:
args.corpus2

'/content/drive/MyDrive/ITStudy/data/[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_train.zip'

In [ ]:
args    # 라이브러리처럼 사용

Namespace(corpus1='/content/drive/MyDrive/ITStudy/data/[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_test.zip', corpus2='/content/drive/MyDrive/ITStudy/data/[└┌┐м╛ю└╠┴ЎAI]▒т╗ч┼╪╜║╞о─л┼╫░э╕о║╨╖∙_train.zip', seed=1234)

In [ ]:
# test 내용 확인 - 맨 앞부터 50줄만
with zipfile.ZipFile(args.corpus1) as z:
    with z.open('test.csv') as f:
        for i, line in enumerate(tqdm(f)):
            if i >= 50 :
                break
            line = line.decode('utf-8').strip()     
            print(line)

0it [00:00, ?it/s]

ID,text
0,"""[기자] 지난 24일 국회 국방위원회에서 현역 대령인 기무부대장이 송영무 국방장관이 계엄 문건을 심각하게 받아들이지 않았다고 폭로성 발언을 했는데요, 기무사령관까지 합세해 국방장관을 공격하면서 공방이 벌어졌었죠. 먼저 당시 화면을 보시겠습니다. [민병삼/기무사 100부대장/지난 24일/국회 국방위원회 : 위수령은 잘못된 것이 아니다.법조계에 문의해보니 최악의 사태를 대비한 계획은 문제가 될 것이 없다고 한다. 장관도 마찬가지 생각이다.] [송영무/국방부장관/지난 24일 : 완벽한 거짓말입니다. 대한민국의 대장까지 마치고 장관하고 있는 사람이 거짓말을 하겠습니까?] 자, 누구 말이 맞는지는 추가 조사를 통해 밝혀야 하겠지만요, 군이라는 조직에서 상관을 그것도 국방장관에 대해 공개적으로 하극상이 벌어진 건 선을 넘어도 너무 넘었다는 지적이 나옵니다. 급기야 대통령이 직접 교통정리에 나섰는데요, 대통령의 입장은 이렇습니다. 이번 사안의 본질은 왜 기무사가 촛불정국에 계엄 실행계획을 세웠는가를 철저히 밝혀내야 것이다. 따라서 국방장관이 계엄 문건을 심각하게 받아들이지 않는 말을 했는지 안 했는지가 핵심은 아니라는 거죠. 그러면서 이번 일을 계기로 기무사 개혁 필요성이 더 커졌다면서, &기무사 개혁안을 서둘러 제출하라&고 지시했습니다. 계엄 문건을 둘러싼 국방장관의 발언 진실 공방은 기무사 개혁안 보고를 받은 뒤에 책임의 경중을 가려 합당한 조처를 취하겠다고도 했는데요, 국방장관의 잘잘못을 따져봐야 한다고도 언급을 한 걸로 볼때 국방장관 경질 가능성도 배제하지 않은 것으로 풀이가 됩니다. 그렇다면 왜 기무사는 이런 사태까지 오도록 한 걸까요? 기무사가 국방 장관 발언을 물고 늘어지는 건 계엄 문건 파동으로 해체 위기에 몰린 기무사 조직 보호를 위한 저항이었다는 시각이 있습니다. 국방 장관이 계엄령 문건 자체를 큰 문제로 인식하지 않았다는 점을 부각시켜서 기무사에 초점이 맞춰진 이번 사안의 본질을 흐리게 하려는 의도가 있었던게 아니냐는 건데요, 실

In [ ]:
# train 내용 확인 - 맨 앞부터 50줄만 
with zipfile.ZipFile(args.corpus2) as z:
    with z.open('train.csv') as f:
        for i, line in enumerate(tqdm(f)):
            if i >= 50 :
                break
            line = line.decode('utf-8').strip()     
            print(line)

0it [00:00, ?it/s]

text,category
이에 따른다면 초등학생들의 소비 문화는 디지털 강국의 ‘개국 공신’이 되려는 몸부림으로 여길 수도 있다. 하지만 지나친 부모의 교육열이 부른 무분별한 소비라는 지적도 만만치 않다. 경기도 수원의 한 초등학교 교사는 “수업 시간에 휴대전화로 문자를 주고받기에 혼냈더니 다음날 부모가 찾아와 ‘내 아이 문제에 끼어들지 말라’고 했다. 요즘 초등학생들은 서구적 가치관도 전통적 가치관도 지니지 못한 경우가 많다”고 고민을 토로했다. 사진 5 BOX ▲ 요즘 아프트 놀이터에서 초등학생을 찾기는 쉽지 않다. 지금 아이들은 학원과 PC방을 전전하고 있는지도 모른다. 요즘 취학하는 아이들은 심각한 주의력 결핍증이 있다고 한다. 유아기 때부터 온갖 시각 영상물에 노출된 탓이다. 다양한 시각 정보를 처리하는 데 익숙하더라도 그 속에서 소비 욕구만 자극받는다면 헛일이다. 인터넷 게임을 넓은 화면에서 즐기고 싶다며 벽걸이형 PDP(Plasma Display Panel) 텔레비전 구입을 요구하는 아이에게 무슨 말을 할 수 있을까. 언제까지 자녀와의 친밀도를 ‘경제력’으로 확인할 수도 없는 일 아닌가. 참교육을 위한 전국 학부모회 장은숙 사무처장은 “올바른 디지털 소비문화 기준을 세워서 적용해야 한다. 적어도 무엇 때문에 필요한지에 대해 ‘식탁 토론’이라도 벌이는 게 중요하다”고 말한다. 글 ▶[한겨레21]바로가기,연예/문화
"""지난 14일 우리정부가 난민 신청을 한 예멘인들에 대해 인도적 체류를 허가했는데요.난민 체류 허가 뒤 맞이한 첫 주말, 서울 도심에선 도로 하나를 사이에 두고난민에 대한 서로 다른 목소리가 터져나왔습니다.강병수 기자가 현장에 다녀왔습니다. [리포트] 정부의 제주도 예멘 난민 일부에 대한 인도적 체류 허가 뒤 맞은 첫 주말. 난민 찬성을 주장하는 집회에서는 인도적 체류만으로는 아직 갈 길이 멀다는 지적이 나왔습니다. [이정원/난민 찬성 집회 참가자 : 예멘 난민들에 대해 난민으로 인정할 수 없다고 결정했습니다. 누구는 난민이고 누구는 난

In [ ]:
# test 라인수 확인
total_count = 0
with zipfile.ZipFile(args.corpus1) as z:  
    with z.open('test.csv') as f: 
       for i, line in enumerate(f) :
           total_count += 1
print(total_count)

# 3104

3104


In [ ]:
# train 라인수 확인
total_count = 0
with zipfile.ZipFile(args.corpus2) as z:  
    with z.open('train.csv') as f: 
       for i, line in enumerate(f) :
           total_count += 1
print(total_count)

# 37649

37649


In [ ]:
print('test 라인 비율 :', round(3104 / (3104 + 37649),2))   
print('trian 라인 비율 :', round(37649 / (3104 + 37649),2))

test 라인 비율 : 0.08
trian 라인 비율 : 0.92


In [ ]:
# test 글자수 확인
total_char = 0
with zipfile.ZipFile(args.corpus1) as z:  # args.corpus 안에 있는 파일 압축을 풉니다
    with z.open('test.csv') as f: # 그 안에 있는 파일을 열었습니다.
       for i, line in enumerate(f) :
           total_char += len(line)
print(total_char)

# 6,354,365

6354365


In [ ]:
# train 글자수 확인
total_char = 0
with zipfile.ZipFile(args.corpus2) as z:  # args.corpus 안에 있는 파일 압축을 풉니다
    with z.open('train.csv') as f: # 그 안에 있는 파일을 열었습니다.
       for i, line in enumerate(f) :
           total_char += len(line)
print(total_char)

# 78,339,064

78339064


# 3. Char Tokenizer

In [ ]:
line = "안녕.나는,만나서!반가워?너는(누구니)?나는\"인공지능'입니다" # 온라인에서 가져온 텍스트를 
line = re.sub('([,.!?()·\"\'])', ' \1 ', line)  # 'r'과 'R'은 문자열에 사용된 이스케이프 시퀀스들을 이스케이프 시퀀스들이 아닌 원시 문자열(raw string)로 만들기 위하여 사용되는 접두사(string prefix)들입니다.
line 